In [1]:
from fastai.text.all import *
import pandas as pd
cols = ["sentiment","text"]
df = pd.read_csv("data/all-data.csv", names =cols, encoding = "ISO-8859-1")
df_test = pd.read_csv("data/public-test-set.csv", names = cols, encoding = "ISO-8859-1")

### Language Model

In [2]:
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning) 
data_lm = TextDataLoaders.from_df(df, is_lm=True, valid_pct = 0.1, text_col='text', label_col="sentiment")

In [3]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3, metrics=[accuracy, Perplexity()], wd=0.1).to_fp16()
#learn.lr_find() #~0.03

In [4]:
learn.fit_one_cycle(4, 3e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.026691,3.309687,0.377413,27.376553,00:06
1,3.356131,3.103397,0.409012,22.273495,00:05
2,2.907588,3.023316,0.421152,20.559362,00:05
3,2.580828,3.020078,0.425073,20.492884,00:05


In [5]:
#learn.lr_find()

In [6]:
learn.unfreeze()
learn.fit_one_cycle(2, 4e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,2.237719,3.071405,0.421444,21.572199,00:06
1,1.912414,3.168513,0.430537,23.772099,00:06


In [7]:
learn.freeze()

In [8]:
TEXT = "The company earned a lot this year and car sales were"
N_WORDS = 50
N_SENTENCES = 2
preds = [learn.predict(TEXT, n_words = N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]
print("\n\n".join(preds))

The company earned a lot this year and car sales were higher than expected . However , the company estimates its net sales for 2007 to remain below the 2009 level . The Group 's balance sheet will have approximately 100 mln euro $ $ 1.7 mln in cash , Alma Media

The company earned a lot this year and car sales were higher than expected . The company 's net sales in 2007 totaled EUR 700 million and it employs some 55 people . Our expertise is a focus core of our core businesses , with one focus on customers , the other us being the core


In [9]:
learn.save_encoder('lm_finetuned')

### Classifier

In [10]:
data_clas = TextDataLoaders.from_df(df, text_col='text', label_col="sentiment", text_vocab=data_lm.vocab)
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.3, metrics=accuracy)
learn = learn.load_encoder('lm_finetuned') #also 'finetuned'

In [11]:
#learn.lr_find() # ~ 2e-2

#### Learning
Gradual unfreezing makes a lot of difference!

In [12]:
learn.fit_one_cycle(3, 2e-2)
#learn.fit(n_epoch=200, lr=2e-2, cbs=EarlyStoppingCallback(monitor='valid_loss', min_delta=0.01, patience=4))

epoch,train_loss,valid_loss,accuracy,time
0,0.808550,0.652849,0.724458,00:02
1,0.666386,0.625067,0.740970,00:02
2,0.577944,0.593710,0.742002,00:02


In [13]:
learn.freeze_to(-2)
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),2e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.642836,0.626299,0.708978,00:02
1,0.525429,0.567122,0.755418,00:03


In [14]:
learn.freeze_to(-3)
learn.fit_one_cycle(2, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.753601,2.823266,0.598555,00:03
1,0.730216,3.338660,0.513932,00:03


In [15]:
#learn.model

In [16]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),2e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.665569,3.512808,0.531476,00:04
1,0.616558,13.639599,0.606811,00:03


In [17]:
learn.freeze()

In [18]:
learn.save_encoder('lm_finished')